In [1]:
import os
import json
from pprint import pprint

from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


In [2]:
data = json.load(open('data/tiny-dev.json'))

pprint(data)

{u'data': [{u'paragraphs': [{u'context': u'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
                             u'qas': [{u'answers': [{u'answer_start': 177,
                                                     u'text': u'Denver Broncos'},
                

                                                    {u'answer_start': 584,
                                                     u'text': u'XXXVIII'}],
                                       u'id': u'56bf28c73aeaaa14008c953c',
                                       u'question': u'What was the first Super Bowl that the Carolina Panthers played in? '},
                                      {u'answers': [{u'answer_start': 219,
                                                     u'text': u'six'},
                                                    {u'answer_start': 138,
                                                     u'text': u'ten'},
                                                    {u'answer_start': 138,
                                                     u'text': u'ten'}],
                                       u'id': u'56bf28c73aeaaa14008c953d',
                                       u'question': u'How many teams can boast a 15\u20131 regular season record?'},
                 

                                                    {u'answer_start': 357,
                                                     u'text': u'Gold'}],
                                       u'id': u'56bf52483aeaaa14008c95c8',
                                       u'question': u'What color were the footballs handed out to a variety of high schools, leading up to Super Bowl 50?'},
                                      {u'answers': [{u'answer_start': 105,
                                                     u'text': u'Golden Super Bowl'},
                                                    {u'answer_start': 100,
                                                     u'text': u'the "Golden Super Bowl"'},
                                                    {u'answer_start': 105,
                                                     u'text': u'Golden Super Bowl'}],
                                       u'id': u'56d70e570d65d2140019833c',
                                       u'question': u'Beca

                             u'qas': [{u'answers': [{u'answer_start': 171,
                                                     u'text': u'Jonathan Stewart'},
                                                    {u'answer_start': 171,
                                                     u'text': u'Jonathan Stewart'},
                                                    {u'answer_start': 180,
                                                     u'text': u'Stewart'}],
                                       u'id': u'56bec8a13aeaaa14008c9435',
                                       u'question': u'Who ran for a Carolina touchdown to make the score 10\u20137?'},
                                      {u'answers': [{u'answer_start': 352,
                                                     u'text': u'Brad Nortman'},
                                                    {u'answer_start': 352,
                                                     u'text': u'Brad Nortman'},
                           

In [3]:
preds_bidaf = json.load(open('predictions_bidaf_prob.json'))

pprint(preds_bidaf)

{u'56be4db0acb8001400a502ec': [u'denver broncos', 0.2304],
 u'56be4db0acb8001400a502ed': [u'denver broncos', 0.2057],
 u'56be4db0acb8001400a502ee': [u"levi's stadium", 0.1964],
 u'56be4db0acb8001400a502ef': [u'national football league', 0.0938],
 u'56be4db0acb8001400a502f0': [u'arabic numerals 50', 0.1752],
 u'56be4e1facb8001400a502f6': [u'quarterback cam newton', 0.6254],
 u'56be4e1facb8001400a502f9': [u'eight', 0.8041],
 u'56be4e1facb8001400a502fa': [u'1995', 0.9941],
 u'56be4eafacb8001400a50302': [u'denver linebacker von miller', 0.8649],
 u'56be4eafacb8001400a50303': [u'two', 0.2974],
 u'56be4eafacb8001400a50304': [u'the broncos', 0.6546],
 u'56be5333acb8001400a5030a': [u'cbs', 0.9974],
 u'56be5333acb8001400a5030b': [u'$5 million', 0.9763],
 u'56be5333acb8001400a5030c': [u'british rock group coldplay with special guest performers beyonc\xe9 and bruno mars',
                               0.1785],
 u'56be5333acb8001400a5030d': [u'coldplay', 0.4255],
 u'56be5333acb8001400a5030e': [u'

In [4]:
preds_rnet = json.load(open('predictions_rnet_prob.json'))

pprint(preds_rnet)

{u'56be4db0acb8001400a502ec': [u'denver broncos', 0.241],
 u'56be4db0acb8001400a502ed': [u'denver broncos', 0.4094],
 u'56be4db0acb8001400a502ee': [u'san francisco bay area at santa clara, california',
                               0.0613],
 u'56be4db0acb8001400a502ef': [u'national football league', 0.0914],
 u'56be4db0acb8001400a502f0': [u'golden anniversary', 0.2806],
 u'56be4e1facb8001400a502f6': [u'cam newton', 0.8108],
 u'56be4e1facb8001400a502f9': [u'eight', 0.8268],
 u'56be4e1facb8001400a502fa': [u'1995', 0.9982],
 u'56be4eafacb8001400a50302': [u'denver linebacker von miller', 0.8661],
 u'56be4eafacb8001400a50303': [u'five', 0.0924],
 u'56be4eafacb8001400a50304': [u'the broncos', 0.4295],
 u'56be5333acb8001400a5030a': [u'cbs', 0.6485],
 u'56be5333acb8001400a5030b': [u'$5 million', 0.7985],
 u'56be5333acb8001400a5030c': [u'british rock group coldplay', 0.4145],
 u'56be5333acb8001400a5030d': [u'beyonc\xe9 and bruno mars', 0.328],
 u'56be5333acb8001400a5030e': [u'super bowl 50', 0

 u'56bebcbe3aeaaa14008c9328': [u'jane kim', 0.8113],
 u'56bebcbe3aeaaa14008c9329': [u'january 30', 0.8903],
 u'56bebd713aeaaa14008c932f': [u'santa clara university', 0.6709],
 u'56bebd713aeaaa14008c9330': [u'$2 million', 0.761],
 u'56bebd713aeaaa14008c9331': [u'week-long', 0.7824],
 u'56bebde53aeaaa14008c9335': [u'tuesday', 0.1545],
 u'56bebde53aeaaa14008c9336': [u'tuesday', 0.2436],
 u'56bebde53aeaaa14008c9337': [u'monday evening', 0.2795],
 u'56bebde53aeaaa14008c9338': [u'san jose', 0.6366],
 u'56bebde53aeaaa14008c9339': [u'golden gate bridge', 0.4439],
 u'56bebe873aeaaa14008c933f': [u'super bowl program', 0.422],
 u'56bebe873aeaaa14008c9340': [u'over $40 million', 0.7802],
 u'56bebe873aeaaa14008c9341': [u'committee', 0.1049],
 u'56bebe873aeaaa14008c9342': [u'apple', 0.0918],
 u'56bebe873aeaaa14008c9343': [u'committee', 0.2133],
 u'56bebec43aeaaa14008c9349': [u'25', 0.4848],
 u'56bebec43aeaaa14008c934a': [u'the 50 fund', 0.2609],
 u'56bec0353aeaaa14008c934d': [u'vince lombardi', 0.56

In [5]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [6]:
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [7]:
def evaluate_orig(dataset, predictions):
    output_dict = {}
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
                
                question = qa['question']
                em_yn = metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
#                 em_score = exact_match_score(prediction, ground_truths)
                if em_yn==True:
                    append = 1
                else:
                    append = 0

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    
    print("Exact Match:", exact_match)
    print("F1: ", f1)

In [8]:
def evaluate_new(dataset, predictions):
    output_dict = {}
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']][0]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
                
                question = qa['question']
                em_yn = metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
#                 em_score = exact_match_score(prediction, ground_truths)
                if em_yn==True:
                    append = 1
                else:
                    append = 0

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    
    print("Exact Match:", exact_match)
    print("F1: ", f1)

In [9]:
## Scores on bidaf

dataset_file = 'data/tiny-dev.json'
dataset_json = json.load(open(dataset_file))
dataset = dataset_json['data']

evaluate_new(dataset, preds_bidaf)

Exact Match: 58.3950617284
F1:  65.9768945791


In [10]:
## Scores on RNET

evaluate_new(dataset, preds_rnet)

Exact Match: 56.2962962963
F1:  64.6540487188


In [11]:
def ensemble(preds_bidaf, preds_rnet):
    new_predictions = {}
    for key, value in preds_bidaf.iteritems():
        uuid = key
        ans_bidaf = value[0]
        prob_bidaf = value[1]
        if key in preds_rnet: #find the same key
            val_rnet = preds_rnet[key]
            ans_rnet = val_rnet[0]
            prob_rnet = val_rnet[1]
        else:
            print("key not found in rnet_json")
        
        ## Check which prob is higher and then assign new answer
        if prob_bidaf >= prob_rnet:
            final_answer = ans_bidaf
        else:
            final_answer = ans_rnet
        
        new_predictions[uuid] = final_answer
    return new_predictions
        

In [12]:
new_predictions = ensemble(preds_bidaf, preds_rnet)

In [13]:
# print(new_predictions)

In [14]:
evaluate_orig(dataset, new_predictions)

Exact Match: 60.2469135802
F1:  67.4230848772


### Data for report

In [26]:
def pred_stats(preds_bidaf, preds_rnet):
    bidaf_better = {}
    rnet_better = {}
    for key, value in preds_bidaf.iteritems():
        uuid = key
        ans_bidaf = value[0]
        prob_bidaf = value[1]
        if key in preds_rnet: #find the same key
            val_rnet = preds_rnet[key]
            ans_rnet = val_rnet[0]
            prob_rnet = val_rnet[1]
        else:
            print("key not found in rnet_json")
        
        ## Check which prob is higher and then assign new answer
        if (prob_bidaf-prob_rnet > 0.30) and (ans_bidaf != ans_rnet):
            bidaf_better[key] = [ans_bidaf, prob_bidaf, ans_rnet, prob_rnet]
        if (prob_rnet-prob_bidaf > 0.30) and (ans_bidaf != ans_rnet) :
            rnet_better[key] = [ans_bidaf, prob_bidaf, ans_rnet, prob_rnet]
        
    return bidaf_better, rnet_better

In [27]:
bidaf_better, rnet_better = pred_stats(preds_bidaf, preds_rnet)
print(bidaf_better)

{u'56bf38383aeaaa14008c956c': [u'2,249', 0.4178, u'2,249 yards and nine', 0.0351], u'56bf38383aeaaa14008c956e': [u'manning', 0.667, u'receiver demaryius thomas', 0.0448], u'56beb57b3aeaaa14008c927b': [u'demaryius thomas', 0.4716, u'receiver demaryius', 0.0754], u'56beb7953aeaaa14008c92af': [u'17 seconds', 0.8691, u'17', 0.3486], u'56be59683aeaaa14008c905a': [u'15', 0.7218, u'least 15', 0.0928], u'56d602631c85041400946edb': [u'beyonc\xe9 and bruno mars', 0.5494, u'british rock group coldplay with special guest performers beyonc\xe9 and bruno mars', 0.2091], u'56d602631c85041400946edc': [u'super bowl xlvii and super bowl xlviii', 0.537, u'super bowl 50', 0.1084], u'56bf159b3aeaaa14008c9509': [u'12\u20134', 0.3951, u'12\u20134 record', 0.0612], u'56d71cb60d65d21400198368': [u'super bowl program', 0.3421, u'super bowl 50 host committee', 0.0233], u'56beb4343aeaaa14008c925f': [u'mario addison', 0.5089, u'lineman mario addison', 0.1824], u'56d9c3a6dc89441400fdb7b8': [u'super bowl xii', 0.555

In [28]:
print(rnet_better)

{u'56beb7953aeaaa14008c92ab': [u'pittsburgh', 0.2199, u'pittsburgh steelers', 0.5941], u'56bf57043aeaaa14008c95da': [u'monday evening and re-branded as super bowl opening night', 0.087, u'tuesday', 0.6421], u'56d729180d65d21400198426': [u'25', 0.2598, u'51', 0.6905], u'56bf3e803aeaaa14008c958b': [u'malik jackson', 0.2225, u'defensive', 0.5714], u'56bf5e7b3aeaaa14008c95fb': [u'on the sidelines', 0.4335, u'sidelines', 0.8269], u'56beba103aeaaa14008c92e9': [u'kicker', 0.2562, u'baltimore ravens', 0.7373], u'56bec9e83aeaaa14008c9462': [u'devin funchess', 0.2017, u'stewart', 0.7403], u'56bf21b43aeaaa14008c9526': [u'super bowl xliv', 0.4039, u'xliv', 0.8582], u'56d9c455dc89441400fdb7c5': [u'six-time grammy winner', 0.1119, u'six-time', 0.9578], u'56d9bf70dc89441400fdb77d': [u'walt disney studios', 0.1572, u'fox', 0.6141], u'56bf34293aeaaa14008c9559': [u'arizona cardinals', 0.4246, u'49\u201315', 0.7912], u'56d726b60d65d214001983ec': [u'dove', 0.2568, u'newton', 0.8623], u'56bf738b3aeaaa14008